Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [13]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labelled examples. Given these sizes, it should be possible to train models quickly on any machine.

In [14]:
url = 'http://yaroslavvb.com/upload/notMNIST/'

def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  if force or not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

strRawCompressedTrainSetFilename = maybe_download('notMNIST_large.tar.gz', 247336696)
strRawCompressedTestSetFilename = maybe_download('notMNIST_small.tar.gz', 8458043)

Found and verified notMNIST_large.tar.gz
Found and verified notMNIST_small.tar.gz


Extract the dataset from the compressed .tar.gz file.
This should give you a set of directories, labelled A through J.

In [15]:
s_iNum_classes = 10
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall()
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != s_iNum_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        s_iNum_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
s_strListTrain_folders = maybe_extract(strRawCompressedTrainSetFilename)
s_strListTest_folders = maybe_extract(strRawCompressedTestSetFilename)

notMNIST_large already present - Skipping extraction of notMNIST_large.tar.gz.
['notMNIST_large/A', 'notMNIST_large/B', 'notMNIST_large/C', 'notMNIST_large/D', 'notMNIST_large/E', 'notMNIST_large/F', 'notMNIST_large/G', 'notMNIST_large/H', 'notMNIST_large/I', 'notMNIST_large/J']
notMNIST_small already present - Skipping extraction of notMNIST_small.tar.gz.
['notMNIST_small/A', 'notMNIST_small/B', 'notMNIST_small/C', 'notMNIST_small/D', 'notMNIST_small/E', 'notMNIST_small/F', 'notMNIST_small/G', 'notMNIST_small/H', 'notMNIST_small/I', 'notMNIST_small/J']


---
Problem 1
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.

---

In [ ]:
######################################## SKIP THIS CELL ############################################
from IPython.display import Image
Image(filename='./notMNIST_large/A/Z2xlZXN0ZWFrLnR0Zg==.png')

Now let's load the data in a more manageable format. Since, depending on your computer setup you might not be able to fit it all in memory, we'll load each class into a separate dataset, store them on disk and curate them independently. Later we'll merge them into a single dataset of manageable size.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road. 

A few images might not be readable, we'll just skip them.

In [ ]:
s_iImage_size = 28  # Pixel width and height.
s_fPixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label, insuring you have at least min_num_images."""
  image_files = os.listdir(folder)
  #An ndarray is a (often fixed) multidimensional container of items of the same type and size
  #so here, we're building a 3d array with indexes (image index, x,y), and type float32
  dataset = np.ndarray(shape=(len(image_files), s_iImage_size, s_iImage_size), dtype=np.float32)
  image_index = 0
  #for each image in the current folder (A, B, etc)
  print(folder)
  for image in os.listdir(folder):
    #get the full image path
    image_file = os.path.join(folder, image)
    try:
      #read image as a bunch of floats, and normalize those floats by using pixel_depth 
      image_data = (ndimage.imread(image_file).astype(float) - s_fPixel_depth / 2) / s_fPixel_depth
      #ensure image shape is standard
      if image_data.shape != (s_iImage_size, s_iImage_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      #and put it in the dataset
      dataset[image_index, :, :] = image_data
      image_index += 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  num_images = image_index
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  #data_folders are either the train or test set. folders within those are A, B, etc
  for folder in data_folders:
    #we will serialize those subfolders (A, B, etc), that's what pickling is
    set_filename = folder + '.pickle'
    #add the name of the current pickled subfolder to the list
    dataset_names.append(set_filename)
    #if the pickled folder already exists, skip
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      #call the load_letter function def above 
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        #and try to pickle it
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

s_strListPickledTrainSets = maybe_pickle(s_strListTrain_folders, 45000)
s_strListPickledTestSets = maybe_pickle(s_strListTest_folders, 1800)

---
Problem 2
---------

Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

---

In [ ]:
######################################## SKIP THIS CELL ############################################
#un-serialize first sub-folder of the train set
random_class_id = np.random.randint(0,s_iNum_classes)
unpickled_rnd_train_set = pickle.load(open(s_strListPickledTrainSets[random_class_id]))
#get xy array representing random image
random_img_id = np.random.randint(0,unpickled_train_set.shape[0])
first_img = unpickled_rnd_train_set[random_img_id,:,:]
# checking image shape, it is 28x28 pixels
# print("image %d from class %d with shape %d" %(random_img_id, random_class_id, first_img.shape))
print("image ", random_img_id, " from class ", random_class_id, " with shape ", first_img.shape)
# denormalization, but commented since doesn't change anything for imshow. The way i understand
# this, is that in these images, the each one of the 28x28 pixels is only encoding grayscale, not
# rgb. And the imshow doc says that it can handle grayscale arrays that are normalized
# (http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.imshow). 
# s_fPixel_depth = 255.0  # Number of levels per pixel.
# first_img = first_img*s_fPixel_depth + s_fPixel_depth/2
# print(first_img[0,:])
plt.imshow(first_img)
plt.show()

---
Problem 3
---------
Another check: we expect the data to be balanced across classes. Verify that.

---

In [ ]:
######################################## SKIP THIS CELL ############################################
#cycle through all train and test sets and count how many examples we have? Also need to check
#their mean and variance?
all_counts = np.zeros(s_iNum_classes)
all_means = np.zeros(s_iNum_classes)
all_variances = np.zeros(s_iNum_classes)

#for cur_class_id, cur_class in enumerate(unpickled_all_train_sets):
for cur_class_id in range(s_iNum_classes):
    #we unpickle here a 3d array with shape: image_ids, xs, ys
    unpickled_cur_train_set = pickle.load(open(s_strListPickledTrainSets[cur_class_id]))
    print ("class ", cur_class_id)
    for cur_image_id in range(len(unpickled_cur_train_set)):
#         print ("image", cur_image_id)
        all_counts[cur_class_id] += 1
#         cur_image = unpickled_cur_train_set()
        all_means[cur_class_id] += np.mean(unpickled_cur_train_set[cur_image_id])
        all_variances[cur_class_id] += np.var(unpickled_cur_train_set[cur_image_id])
        
print ("all_counts: %d", all_counts)

all_means = np.divide(all_means, s_iNum_classes)
print ("mean of all_means: ", all_means)

all_variances = np.divide(all_variances, s_iNum_classes)
print ("mean of all_variances: ", all_variances)

Merge and prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune `s_iTrainSize` as needed. The labels will be stored into a separate array of integers 0 through 9.

Also create a validation dataset for hyperparameter tuning.

In [ ]:
#from p_iNb_rows and p_iImg_size: 
#  return dataset:  an empty 3d array that is [p_iNb_rows, p_iImg_size, p_iImg_size]
#  return labels: an empty vector that is [p_iNb_rows]
def make_arrays(p_iNb_rows, p_iImg_size):
  if p_iNb_rows:
    dataset = np.ndarray((p_iNb_rows, p_iImg_size, p_iImg_size), dtype=np.float32)
    labels = np.ndarray(p_iNb_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

#p_strListPickle_files is an array containing the filenames of the pickled data
def merge_datasets(p_strListPickle_files, p_iTrainSize, p_iValidSize=0):
  #iNum_classes is the number of filenames/folders
  iNum_classes = len(p_strListPickle_files)
  #make empty arrays for validation and training sets and labels
  valid_dataset, valid_labels = make_arrays(p_iValidSize, s_iImage_size)
  train_dataset, train_labels = make_arrays(p_iTrainSize, s_iImage_size)
  #number of items per class. // is an int division in python3, not sure in python2
  iNbrOfValidItemsPerClass = p_iValidSize // iNum_classes
  iNbrOfTrainItemPerClass = p_iTrainSize // iNum_classes
  
  #for each file in p_strListPickle_files
  iStartValidId, iStarTrainId = 0, 0
  iEndValidId, iEndTrainId = iNbrOfValidItemsPerClass, iNbrOfTrainItemPerClass
  iEndListId = iNbrOfValidItemsPerClass+iNbrOfTrainItemPerClass
  for iPickleFileId, strPickleFilename in enumerate(p_strListPickle_files):       
    try:
      with open(strPickleFilename, 'rb') as f:
        letter_set = pickle.load(f)
        # let's shuffle the letters to have random validation and training set
        np.random.shuffle(letter_set)
        
        #if we asked for a validation set
        if valid_dataset is not None:
          #the first iNbrOfValidItemsPerClass items in letter_set are used for the validation set
          valid_letter = letter_set[:iNbrOfValidItemsPerClass, :, :]
          valid_dataset[iStartValidId:iEndValidId, :, :] = valid_letter
          valid_labels[iStartValidId:iEndValidId] = iPickleFileId
          start_v += iNbrOfValidItemsPerClass
          end_v += iNbrOfValidItemsPerClass
                    
        #the rest of the items are used for the training set
        train_letter = letter_set[iNbrOfValidItemsPerClass:iEndListId, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = iPickleFileId
        start_t += tsize_per_class
        end_t += tsize_per_class
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise    
  return valid_dataset, valid_labels, train_dataset, train_labels

#original values            
# s_iTrainSize = 200000
# s_iValid_size = 10000
# s_iTestSize = 10000
s_iTrainSize = 2000
s_iValid_size = 100
s_iTestSize = 100

#call merge_datasets on data_sets and labels
valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(s_strListPickledTrainSets, s_iTrainSize, s_iValid_size)
_,          _,               test_dataset,  test_labels  = merge_datasets(s_strListPickledTestSets, s_iTestSize)

#print shapes for data sets and their respective labels. data sets are 3d arrays with [image_id,x,y] and labels
#are [image_ids]
print('Training:',   train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:',    test_dataset.shape,  test_labels.shape)

Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [ ]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

---
Problem 4
---------
Convince yourself that the data is still good after shuffling!

---

Finally, let's save the data for later reuse:

In [ ]:
pickle_file = 'notMNIST.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [ ]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

---
Problem 5
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.

Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---

---
Problem 6
---------

Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!

---